In [1]:
import numpy as np
import matplotlib.pyplot as plt
#% inline
%matplotlib
import PIL.Image
import networkx as nx
import pandas as pd
import string

from functions.genome_topology import open_pdb
from functions.genome_topology import select_chrom
from functions.genome_topology import geom_distance
from functions.genome_topology import make_graph
from functions.genome_topology import fractal_dimension
from functions.genome_topology import get_matrix
from functions.genome_topology import normalize_psc
from functions.plotting_tools import set_layout
from functions.plotting_tools import matrix_plot

Using matplotlib backend: <object object at 0x103563450>


## CIRCUIT TOPOLOGY ANALISIS OF CHROMOSOME 3D STRUCTURES

This notebook processes 3D structures of chromosomes (in PDB format) to extract topological parameters such as:
- CT parameters (percentage of series, parallel and cross, number of contacts) 
- Network clustering coefficient
- Fractal dimension (calculated over the CT topology matrix)

First, the PDB structures are processed in order to find spatial contacts. A spatial cutoff needs to be defined for contact selection (suggested range of values: 1.0-2.0). Chromosome ID is expressed as letters (ex: 'chr a'). It is possible to save the topology matrix in TIF format, and the topological parameters in csv format.

Choose paramters: cell number, cutoff radius and number of neighbors to exclude from the analysis

In [2]:
path = 'data/pdb/Cell_ID-01'
r_cutoff=1.0
neighbours=1
set_layout()
n_all_chr=23
letters=list(string.ascii_lowercase)
chr_vec=['chr {}'.format(letter) for letter in letters[:n_all_chr]]
cell=path[-10:]

print('analyzing {}'.format(cell))

analyzing Cell_ID-01


### PROCESS SINGLE CHROMOSOME

In [3]:
save_data= True
save_matrix= True
path_savematrix='results/matrices'
path_savedata='results/CT parameters'

In [4]:
#Choose chromosome
chosen_chr=2 

#Find contacts
n, coord= select_chrom(chosen_chr, path)
dist, n_contact, index=geom_distance(coord, r_cutoff, neighbours) 

#Extract topological parameters
mat, stats = get_matrix(index,chr_vec[chosen_chr])
P, S, X=normalize_psc(stats, len(index))
Dim_fractal, r2=fractal_dimension(mat, plot_fig=0)

#Build network and calculate clustering coefficient
G=make_graph(index)
clustering= nx.average_clustering(G)

#plot figures
#Plot 2D projections of chromosome choordinate
plt.figure()
plt.plot(coord[:,0], coord[:,2])
plt.title('CHROMOSOME {}'.format(chr_vec[chosen_chr][-1:].upper()))
plt.xlabel('y')
plt.ylabel('z')

#Plot topology matrix
plt.figure(figsize=(7, 7))
plt.imshow(mat)
plt.show()
# PIL.Image.fromarray(mat.astype(np.uint8)).save("{}/top_matrix_{}_{}.tif".format(path_savematrix, cell, chr_vec[chosen_chr]))


KeyboardInterrupt: 

In [ ]:
plt.figure()
matrix_plot(mat, 'chrom {}'.format(chosen_chr))

plt.savefig('chrom{}_mat.pdf'.format(chosen_chr))


# PROCESS ALL CHROMOSOMES IN A CELL

Set path for printing results

In [5]:
save_data= True
save_matrix= False
plot_matrix=True
#path_savematrix='results/matrices/{}'.format(cell)
path_savematrix='results/matrices'
path_savedata='results/CT parameters'

Loop over all chromosomes in a cell

In [6]:
P=np.zeros(n_all_chr)
S=np.zeros(n_all_chr)
X=np.zeros(n_all_chr)
N_contacts=np.zeros(n_all_chr)
Dim_fractal=np.zeros(n_all_chr)
r2_fractalfit=np.zeros(n_all_chr)
clustering=np.zeros(n_all_chr)

In [11]:
import concurrent.futures
import os

with concurrent.futures.ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = []
    all_chromosomes_data = []
    for i in range(1,25):  # Loop over cells
        cell = f"Cell_ID-{i:02}"
        print('analyzing the',cell)
        path = f"data/pdb/{cell}"
        for t in range(n_all_chr):
            print('analyzing chromosome {}'.format(chr_vec[t]))
            n, coord= select_chrom(t, path)
            dist, n_contact, index=geom_distance(coord, r_cutoff, neighbours)
            mat, stats = get_matrix(index,chr_vec[0])
            P[t], S[t], X[t]=normalize_psc(stats, len(index))
            N_contacts[t]=len(index)
            Dim_fractal[t], r2_fractalfit[t]=fractal_dimension(mat, plot_fig=0)
            G=make_graph(index)
            clustering[t]= nx.average_clustering(G)
            
            if plot_matrix:
                plt.figure(figsize=(7, 7))
                plt.imshow(mat)
                plt.title('CHROMOSOME {}'.format(chr_vec[t][-1:].upper()))
                plt.show()
            
            if save_matrix:
                PIL.Image.fromarray(mat.astype(float)).save("{}/top_matrix_{}_{}.tif".format(path_savematrix,
                                                                                        cell, chr_vec[t]))
        if save_data:
                topology_parameters = {'Sample':cell,'Chromosome':chr_vec[t],'Parallel (%)':P, 'Series (%)':S, 'Cross (%)':X, 'N contacts': N_contacts,
                                    'Fractal dimension':Dim_fractal, 'r squared': r2_fractalfit, 
                                    'Clustering': clustering}
                
                topology_parameters= pd.DataFrame(topology_parameters)
                
                topology_parameters.to_csv('{}/Top_parameters_{}.csv'.format(path_savedata, cell))
                print("Data saved in {}/Top_parameters_{}.csv".format(path_savedata, cell))



analyzing the Cell_ID-01
analyzing chromosome chr a


In [ ]:
plt.figure()
matrix_plot(mat, 'Cell_ID-01')
plt.show()
plt.savefig('/Users/jialechen/Desktop/PhD/CT/Pang_2022_GenomeBiol_3D/results/Cell_ID_01_mat.pdf')
